In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import torch
import pickle
import keyboard
from utils.es_utils import *
from utils.env_utils import *
from utils.eval_utils import *
from utils.train_utils import *
from models.Buffer import Buffer, Pool
from models.Discriminator import Discriminator
from models.DeepGenerator import GeneratorPolicy
from marl_aquarium import aquarium_v0

In [ ]:
couzin_path = rf"..\data\processed\couzin_interactions"